In [2]:
import sys, os
sys.path.insert(1,os.path.abspath('../src'))

Import our code under test.

In [3]:
from fpipeline import pipeline, stepfn, conditionfn, variables

## Test Framework
    

In [4]:
from dataclasses import dataclass, field
from IPython.display import display
from typing import Callable

@dataclass
class Test:
    name: str
    result: any
    negated: bool = False
    def _success(self):
        display({"text/html": f"✅: {self.name}"}, raw=True)
    def _failure(self):
        display(
            {"text/html": f"<span style='color:red'>❌: {self.name}</span>"}, raw=True)
    def _check(self, pred: Callable[[any, any], bool], value: any):
        if bool(pred(self.result, value)) ^ self.negated:
            self._success();
        else:
            self._failure()
    def equals(self, value: any):
        self._check(lambda a, b: a == b, value)
    def is_same(self, value: any):
        self._check(lambda a, b: a is b, value)
    def is_true(self):
        self._check(lambda a, b: not not a, True)
    def is_false(self):
        self._check(lambda a, b: not a, False)
    @property
    def negate(self):
        return Test(self.name, self.result, negated=not self.negated)
    @property
    def exec(self):
        try:
            return Test(self.result())
        except Exception as ex:
            return Test(ex)
    def is_exception(self):
        self._check(isinstance, Exception)
    def isinstance(self, cls):
        self._check(isinstance, cls)

In [5]:
Test("Test Test", None).equals(None)

✅: Test Test

In [6]:
Test("Test Negate Test", None).negate.equals(False)

✅: Test Negate Test

## Test Data Setup

In [7]:
@dataclass
class ctx:
    value: any = None
    trace: list[tuple] = field(default_factory=list)
    def __post_init__(self):
        trace = list()

@stepfn
def tstep(data: ctx, *args, **kwargs):
    d = (*args, {**kwargs})
    data.trace.append(d)
    return data

@conditionfn
def has_value(data: ctx, value):
    return data.value == value

In [8]:
Test("@stepfn step should take a single argument",
    tstep(7, 8).__code__.co_argcount
).equals(1)

✅: @stepfn step should take a single argument

In [9]:
Test("@conditionfn condition should take a single argument",
    has_value(7, 8).__code__.co_argcount
).equals(1)

✅: @conditionfn condition should take a single argument

In [10]:
Test("@stepfn capture args",
    tstep(7, 5)(ctx()).trace
).equals([(7, 5, {})])

✅: @stepfn capture args

In [14]:
Test("@conditionfn capture args (true)",
    has_value(7)(ctx(7))
).is_true()

✅: @conditionfn capture args (true)

In [13]:
Test("@conditionfn capture args (false)",
    has_value(7)(ctx())
).is_false()

✅: @conditionfn capture args (false)

In [ ]:
Test("Simple pipeline",
    pipeline(tstep(7), tstep(78))(ctx()).trace
 ).equals([(7, {}), (78, {})])

✅: Simple pipeline

In [ ]:
def f1():
    with variables(ctx()) as vars:
        return vars.variables
Test("With no variables",
    f1()
).equals({})

✅: With no variables